In [ ]:
import zipfile
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
#import dataset files from google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = '/content/drive/MyDrive/ML2024_HW2_data'

In [ ]:
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

In [ ]:
augmented_train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='sparse'
)

train_generator_2 = augmented_train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='sparse'
)

In [ ]:
def create_model_1():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) #optimizer in model 2 is to be changed
    return model


In [ ]:
def create_model_2():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(5, activation='softmax')
    ])
    model.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy']) #optimizer in model 2 is to be changed
    return model

In [ ]:
class_counts = {
    0: 1000,
    1: 1500,
    2: 1500,
    3: 2000,
    4: 369
}

classes = np.array(list(class_counts.keys()))
counts = np.array(list(class_counts.values()))

class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=np.concatenate([
    np.full(v, k) for k, v in class_counts.items()
]))

class_weight_dict = dict(zip(classes, class_weights))
print("Wagi klas:", class_weight_dict)

In [ ]:
model_1 = create_model_1()
history_1 = model_1.fit(train_generator, validation_data=test_generator, epochs=10, class_weight=class_weight_dict)

In [ ]:
model_2 = create_model_2()
history_2 = model_2.fit(train_generator_2, validation_data=test_generator, epochs=10, class_weight=class_weight_dict)

In [ ]:
print(model_1.summary())

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns


In [ ]:
print(train_generator.class_indices)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(),
                yticklabels=test_generator.class_indices.keys())
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    print(f"{title} (text):")
    print(cm)
    print()

def plot_training_history(history_1, history_2):
    train_acc_1 = history_1.history['accuracy']
    val_acc_1 = history_1.history['val_accuracy']
    train_loss_1 = history_1.history['loss']
    val_loss_1 = history_1.history['val_loss']

    train_acc_2 = history_2.history['accuracy']
    val_acc_2 = history_2.history['val_accuracy']
    train_loss_2 = history_2.history['loss']
    val_loss_2 = history_2.history['val_loss']

    epochs = range(1, len(train_acc_1) + 1)

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_acc_1, label='Model_1 Train Accuracy', marker='o')
    plt.plot(epochs, val_acc_1, label='Model_1 Val Accuracy', marker='o')
    plt.plot(epochs, train_acc_2, label='Model_2 Train Accuracy', marker='s')
    plt.plot(epochs, val_acc_2, label='Model_2 Val Accuracy', marker='s')
    plt.title('Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_loss_1, label='Model_1 Train Loss', marker='o')
    plt.plot(epochs, val_loss_1, label='Model_1 Val Loss', marker='o')
    plt.plot(epochs, train_loss_2, label='Model_2 Train Loss', marker='s')
    plt.plot(epochs, val_loss_2, label='Model_2 Val Loss', marker='s')
    plt.title('Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

y_true = test_generator.classes
y_pred_1 = np.argmax(model_1.predict(test_generator), axis=1)
y_pred_2 = np.argmax(model_2.predict(test_generator), axis=1)

accuracy_1 = accuracy_score(y_true, y_pred_1)
f1_1 = f1_score(y_true, y_pred_1, average='weighted')
precision_1 = precision_score(y_true, y_pred_1, average='weighted')
recall_1 = recall_score(y_true, y_pred_1, average='weighted')

accuracy_2 = accuracy_score(y_true, y_pred_2)
f1_2 = f1_score(y_true, y_pred_2, average='weighted')
precision_2 = precision_score(y_true, y_pred_2, average='weighted')
recall_2 = recall_score(y_true, y_pred_2, average='weighted')

plot_training_history(history_1, history_2)

plot_confusion_matrix(y_true, y_pred_1, "Confusion Matrix: Model 1")
plot_confusion_matrix(y_true, y_pred_2, "Confusion Matrix: Model 2")

print("Classification: Model 1")
print(classification_report(y_true, y_pred_1, target_names=test_generator.class_indices.keys()))

print("Classification: Model 2")
print(classification_report(y_true, y_pred_2, target_names=test_generator.class_indices.keys()))

print(f"Model 1 - Accuracy: {accuracy_1:.2f}, F1 Score: {f1_1:.2f}, Precision: {precision_1:.2f}, Recall: {recall_1:.2f}")
print(f"Model 2 - Accuracy: {accuracy_2:.2f}, F1 Score: {f1_2:.2f}, Precision: {precision_2:.2f}, Recall: {recall_2:.2f}")


